# Practical 1

## Characterizing your mystery genome

The purpose of today's practical is to characterize your mystery genome to learn more about it's age, preparation method and quality. 

You will use a combination of custom scripts and published tools, including `samtools` (https://www.htslib.org/doc/samtools.html)[1] and `PMDtools` (https://github.com/pontussk/PMDtools)[2], to analyze your mystery genome. 

### Getting Started

<b>If you haven't already done so, start an interactive session</b>

- Sign in to https://ood.huit.harvard.edu/ 
- Navigate to `Interactive Apps → Jupyter Lab - HEB 115`
- Launch a Jupyter Lab session with the following parameters:
    - Number of hours: 2
    - Number of CPUs: 1
- When the session is ready, click “Connect to Jupyter”

<b>Create a working directory (called "practical_1" from which you will run commands and store any files that you generate</b>

```bash
mkdir practical_1
cd practical_1
```

<b>Copy these practical instructions to your working directory and open them as a Jupyter Notebook</b>

```bash
cp ~/153784/practical_instructions/Practical1.ipynb ./
```

Then navigate to the practical_1 directory on the sidebar and click on Practical1.ipynb to open it as a Jupyter Notebook

### Part 1) Calculate the read length distribution for your mystery genome

First we will use `samtools` and some simple `awk` and `bash` scripting to quantify the distribution of DNA segment lengths (or read lengths) in your mystery genome. 

In order to save time, we will only consider the first 1 million reads in your bam file. 

The following command will extract the first 1 million reads from your bam file, calculate  the length of each read, and then create a file called `read_length_distribution.txt` that reports the number of sequences of each length in your bam file. 

<i><b>Tip</b>: Make sure that you provide the full path to your mystery genome's bam file (i.e., include information about the folder where it is located, not just the name of your bam)</i>

```bash
samtools view {POINTER TO YOUR MYSTERY BAM FILE} | head -n 1000000 | awk '{print length($10)}' | sort | uniq -c > read_length_distribution.txt
```

-----------------------
Next, the following command will format your results into an easy to read table called `read_length_table.txt` that has appropriate headers

```bash
awk 'BEGIN {print "Read_Length\tCount"} {print $2 "\t" $1}' read_length_distribution.txt | sort -n -k 1 > read_length_table.txt
```

-----------------------
Take a peek at the `read_length_table.txt` you just produced and see if it gives you an idea about the age of your mystery genome. Does the distribution seem more consistent with that of an ancient or present-day individual?

You can proceed to part 2 for now, but in your final report, use the information in `read_length_table.txt` to create a histogram that displays your read length distribution similar to the one shown in class. 

<i><b>Tip</b>: You can use any tool of your choosing to create your plot, including python (in which case you can make it right here in this Jupyter Notebook), R, or even Excel or Google sheets. If you aren't sure where to start, try asking ChatGPT or another AI tool! </i>

### Part 2) Check whether your mystery genome shows signatures of ancient DNA damage (C-to-T misincorporations)

In this step, you will look for ancient DNA damage patterns in your mystery genome. To do this, you’ll need to use a special version of your BAM file in which the reads have not been “soft-clipped.”

Soft-clipping is when the CIGAR string in a BAM file marks the first and/or last few bases of each read to be ignored in the alignment. In ancient DNA studies, this is often done because damage is most likely to occur at the ends of molecules, and we don’t want those damaged bases to bias population genetic analyses.

The number of soft-clipped bases depends on the type of UDG treatment applied to the library:

- UDG-plus: 0 bases clipped (no C-to-T damage expected)
- UDG-half: 2 bases clipped (damage only expected in the first 1–2 bases at each end)
- UDG-minus: 10 bases clipped (damage expected to extend further into the molecule)

The mystery genome you worked with in the previous class (and that you will continue to use for all other analyses) was already soft-clipped. For this part of today's exercise, however, you need to use the unclipped version of the same genome so that you can actually see the damage signal.

You can find the unclipped BAMs here:
`~/153784/data/mystery_genomes/unclipped_genomes`



-----------------------
To assess ancient DNA damage patterns in your unclipped bam file, we will use `PMDtools`, a handy python-based package for analyzing ancient DNA data. To save time, we will only consider the first 1 million reads in your bam file. 

To run PMDtools use the following command:

```bash
samtools view {POINTER TO YOUR UNCLIPPED MYSTERY BAM FILE} | head -1000000 | python ~/153784/tools/PMDtools/pmdtools.0.60.py --deamination > damage_pattern_table.txt
```

-----------------------
<i><b>Note (this may apply to some of you):</b> 
If you get an error like this:<br>
  `cigar found:',cigar,'PMDtools only supports cigar operations M, I, S and D, the alignment has been excluded`<br>
<i>it means that some of the reads in your BAM file are hard-clipped rather than soft-clipped.
  - <i>Soft-clipping (S) keeps the clipped bases in the read but marks them to be ignored.
  - <i>Hard-clipping (H) actually removes the bases and records that fact in the CIGAR string.

<i>PMDtools can’t handle hard-clipped reads, so it crashes when it sees them.

<i>To get around this, filter out any reads with an H in their CIGAR string (the 6th column in the BAM). You can do this by adding the following extra `awk` step between the `samtools` and `head` commands: `awk '$6 !~ /H/'` Make sure to add an extra pipe symbol (|) so that you are clearly differentiating between each step. </i> 

-----------------------

When it is finished running, `PMDtools` should have created a file called `damage_pattern_table.txt` which contains information about the rate of misincoroporations at the first and last 30 bases in each read.
- <b>Note</b>: For the first 30 bases, it reports the rate of misincorporations at positions where it was expecting to see a "C", while for the last 30 bases it reports the reate of misincorporations at positions where it was expecting to see a "G"

This table isn't formatted in a very easy to read way, so you can use the following custom script, which uses a combination of `awk` commands to reformat it:

```bash
bash ~/153784/tools/custom_scripts/reformat_pmdtools_output.sh damage_pattern_table.txt reformatted_damage_pattern_table.txt
```

-----------------------
You just created a more nicely formatted table called `reformatted_damage_pattern_table.txt` which contains information about the rate of misincorporations in the first and last 30 bases in each read. Take a look at this table, paying special attention to the C->T and G->A misincorporations.

Using the information you gathered from this table and from the previous read length distribution analysis, what conclusions can you draw about your mystery genome? 
- Do you think your mystery genome is from an ancient or present-day individual?
- If they are ancient, do you think the DNA sample underwent any UDG treatment during library preparation? 

Once you have answered the above question, you can proceed to part 3, but for your final report, use the information in `reformatted_damage_pattern_table.txt` to create a "smiley" plot that displays the misincorporation rates in your mystery genome, similar to the one shown in class, using a plotting tool of your choosing:

<i><b>Note:</b> Make sure that you don't zoom too far in on the Y-axis. You should see the pattern best with a maximum Y-axis value of 20-30%.</i>

### Part 3) Calculate the coverage of your mystery genome on different SNP sets

Finally, let's calculate the coverage of your mystery genome on three different SNP sets in order to try to determine whether your mystery genome underwent targeted enrichment capture or genome-wide ("shotgun") sequencing. 

If your mystery genome underwent targeted enrichment capture, its coverage on the positions that were enriched for will be signficantly higher than the autosomal coverage. 

So let's check your mystery genome's coverage on the following three SNP sets:
- <b>Autosomes</b>: For samples that underwent 'shotgun' sequencing, instead of targeted enrichment capture, we report the coverage across all of the autosomes. That's the coverage across all positions on chromosomes 1-22 in the human genome. 
- <b>1240k SNP set</b>: The '1240k' array is a set of approximately 1.24 million SNPs that are included on a targeted enrichment capture array that is widely used by ancient DNA researchers. 
- <b>390k SNP set</b>: The '390k' array is a list of approximately 390,000 SNPs that were included on a targeted enrichment capture array that pre-dated the 1240k array. A small number of ancient genomes underwent targeted enrichment capture on this SNP set.

-----------------------

You'll use the function `samtools depth` to cacluate the coverage of your mystery genome on these three SNP sets. 
- We'll specify the SNP sets to focus on by providing a SNP list with the `-b` parameter.
- In all cases, we'll use the `-a` parameter to report counts at all of the SNPs of interest, not just those that have at least one overlapping read.

Use the following three commands to calculate coverage at each SNP set. These can take a while to run, so we'll submit each of them as a separate job to the compute cluster:

<b>Compute coverage on the autosomes:</b>

```bash
sbatch --wrap="samtools depth --min-MQ 10 --min-BQ 20 -b ~/153784/data/reference_data/autosomal_positions.bed -a {POINTER TO YOUR MYSTERY BAM FILE} | awk '{sum+=\$3} END { print sum/NR}' > coverage_autosomes.txt"
```

<b>Compute coverage on the 1240k SNP set:</b>

```bash
sbatch --wrap="samtools depth --min-MQ 10 --min-BQ 20 -b ~/153784/data/reference_data/1240k_positions.bed -a {POINTER TO YOUR MYSTERY BAM FILE} | awk '{sum+=\$3} END { print sum/NR}' > coverage_1240k.txt"
```

<b>Compute coverage on the 390k SNP set:</b>

```bash
sbatch --wrap="samtools depth --min-MQ 10 --min-BQ 20 -b ~/153784/data/reference_data/390k_positions.bed -a {POINTER TO YOUR MYSTERY BAM FILE} | awk '{sum+=\$3} END { print sum/NR}' > coverage_390k.txt" 
```

## When you are finished

### Be sure to include the following in your report: 
<b>Methods section</b>: <br>
A description of each of the analyses that you performed. 

<b>Results section</b>: <br>
Make sure that your results section includes the following results:
- Figures showing the “smiley” plot from part 1 and the histogram from part 2.
- Report (in words) the coverage you calculated in part 3. 

<b>Conclusion section</b>: <br>
Be sure to address the following questions. Based on the results of your analyses: 
- Do you think your mystery genome was sequenced from an ancient or present-day individual?
- Do you think your mystery genome underwent targeted enrichment capture or shotgun sequencing? If it underwent capture, what array do you think was used?
- If the individual was ancient, what type of UDG treatment do you think was used during processing (minus, plus or half)? 

### Additional Questions to answer at the end of your report: 
1) In part 1, what is the purpose of the awk statement `awk '{print length($10)}'`? 
    - <i><b>Tip:</b> Try using the head command to view the input that the samtools view command is passing to this `awk` statement. <i>
2) In part 2, why does `PMDtools` output include more than just the rate of C-to-T misincorporations as part of this analysis? What other misincorporation rate may be impacted by ancient DNA damage?
3) In part 2 you used a different version of your mystery genome that had not undergone "soft-clipping." Based on the C-to-T misincorporation rate you observed, how many bases do you think were soft-clipped from the terminal ends of each DNA molecule in the soft-clipped version of your mystery genome that you are using for all other analyses in this course?
4) In part 3, what do the parameters `--min-MQ` and `--min-BQ` specify in the `samtools depth` function? 
    - <i><b>Tip:</b> Check out the `samtools depth` documentation:</i> https://www.htslib.org/doc/samtools-depth.html


## References

1) Danecek, P, et al. (2021) Twelve years of SAMtools and BCFtools. Gigascience 10.2 :giab008.
2) P Skoglund, BH Northoff, MV Shunkov, A Derevianko, S Pääbo, J Krause, M Jakobsson (2014): Separating ancient DNA from modern contamination in a Siberian Neandertal, Proceedings of the National Academy of Sciences USA